In [1]:
import pandas as pd
from utils import *
from mappings import *

In [2]:
df = read_dbf('../data/raw/peaks.DBF')

In [3]:
df.head()

,peakid,pkname,pkname2,location,heightm,heightf,himal,region,open,unlisted,...,peakmemo,pyear,pseason,pexpid,psmtdate,pcountry,psummiters,psmtnote,refermemo,photomemo
0,AMAD,Ama Dablam,Amai Dablang,Khumbu Himal,6814,22356,12,2,True,False,...,"Other map altitudes:\r\n 6814m - HMG-MT, HMG...",1961,1,AMAD61101,Mar 13,"New Zealand, USA, UK","Mike Gill, Wally Romanes, Barry Bishop, Michae...",None,None,W Face (High 126:5 May 1993)\r\nSE Face (High ...
1,AMPG,Amphu Gyabjen,Amphu Gyabien,Khumbu Himal (N of Ama Dablam),5630,18471,12,2,True,False,...,"Other map altitudes:\r\n 5630m - HMG-Finn, N...",1953,1,AMPG53101,Apr 11,UK,"John Hunt, Tom Bourdillon",None,None,None
2,ANN1,Annapurna I,None,Annapurna Himal,8091,26545,1,5,True,False,...,"Other map altitudes:\r\n 8091m - HMG-MT, HMG...",1950,1,ANN150101,Jun 03,France,"Maurice Herzog, Louis Lachenal",None,Dyhrenfurth history 1950-1977 (MM 58:44-47 Nov...,S Face (High 122:3 Jan 1993) (Beghin accident)...
3,ANN2,Annapurna II,None,Annapurna Himal,7937,26040,1,5,True,False,...,"Other map altitudes:\r\n 7937m - HMG-MT, HMG...",1960,1,ANN260101,May 17,"UK, Nepal","Richard Grant, Chris Bonington, Ang Nyima Sherpa",None,Dyhrenfurth history 1960-1976 (MM 51:36-37 Sep...,N Face (MM 51:36 Sep 1976)
4,ANN3,Annapurna III,None,Annapurna Himal,7555,24787,1,5,True,False,...,"Other map altitudes:\r\n 7555m - HMG-MT, HMG...",1961,1,ANN361101,May 06,India,"Mohan S. Kohli, Sonam Gyatso, Sonam Girmi",None,None,S Side (MM 125:11 Jan 1989)\r\nSW Face (MM 71:...


In [26]:
df_mountains = pd.DataFrame.from_dict(himal_map, orient='index', columns=['name'])
df_regions = pd.DataFrame.from_dict(region_map, orient='index', columns=['name'])

In [6]:
df['host'] = apply_map(df.phost, peak_host_map)
df.drop('phost', axis=1, inplace=True)

In [7]:
df.head()

,peakid,pkname,pkname2,location,heightm,heightf,himal,region,open,unlisted,...,pyear,pseason,pexpid,psmtdate,pcountry,psummiters,psmtnote,refermemo,photomemo,host
0,AMAD,Ama Dablam,Amai Dablang,Khumbu Himal,6814,22356,12,Khumbu-Rolwaling-Makalu,True,False,...,1961,1,AMAD61101,Mar 13,"New Zealand, USA, UK","Mike Gill, Wally Romanes, Barry Bishop, Michae...",None,None,W Face (High 126:5 May 1993)\r\nSE Face (High ...,Nepal
1,AMPG,Amphu Gyabjen,Amphu Gyabien,Khumbu Himal (N of Ama Dablam),5630,18471,12,Khumbu-Rolwaling-Makalu,True,False,...,1953,1,AMPG53101,Apr 11,UK,"John Hunt, Tom Bourdillon",None,None,None,Nepal
2,ANN1,Annapurna I,None,Annapurna Himal,8091,26545,1,Annapurna-Damodar-Peri,True,False,...,1950,1,ANN150101,Jun 03,France,"Maurice Herzog, Louis Lachenal",None,Dyhrenfurth history 1950-1977 (MM 58:44-47 Nov...,S Face (High 122:3 Jan 1993) (Beghin accident)...,Nepal
3,ANN2,Annapurna II,None,Annapurna Himal,7937,26040,1,Annapurna-Damodar-Peri,True,False,...,1960,1,ANN260101,May 17,"UK, Nepal","Richard Grant, Chris Bonington, Ang Nyima Sherpa",None,Dyhrenfurth history 1960-1976 (MM 51:36-37 Sep...,N Face (MM 51:36 Sep 1976),Nepal
4,ANN3,Annapurna III,None,Annapurna Himal,7555,24787,1,Annapurna-Damodar-Peri,True,False,...,1961,1,ANN361101,May 06,India,"Mohan S. Kohli, Sonam Gyatso, Sonam Girmi",None,None,S Side (MM 125:11 Jan 1989)\r\nSW Face (MM 71:...,Nepal


In [8]:
df.drop(
	['peakmemo', 'pyear', 'pseason', 'pexpid', 'psmtdate', 'pcountry', 'psummiters', 'psmtnote', 'heightf'], axis=1, inplace=True)

In [9]:
locations_df = df[['location']]\
	.drop_duplicates(ignore_index=True)\
	.rename({'location': 'name'}, axis=1)\
	.reset_index(names='id')

In [10]:
df = df.merge(locations_df, how='left', left_on='location', right_on='name')\
	.rename({'id': 'location_id'}, axis=1)\
	.drop(['location', 'name'], axis=1)

In [11]:
df.head()

,peakid,pkname,pkname2,heightm,himal,region,open,unlisted,trekking,trekyear,restrict,pstatus,refermemo,photomemo,host,location_id
0,AMAD,Ama Dablam,Amai Dablang,6814,12,Khumbu-Rolwaling-Makalu,True,False,False,None,None,2,None,W Face (High 126:5 May 1993)\r\nSE Face (High ...,Nepal,0
1,AMPG,Amphu Gyabjen,Amphu Gyabien,5630,12,Khumbu-Rolwaling-Makalu,True,False,False,None,Opened in 2002,2,None,None,Nepal,1
2,ANN1,Annapurna I,None,8091,1,Annapurna-Damodar-Peri,True,False,False,None,None,2,Dyhrenfurth history 1950-1977 (MM 58:44-47 Nov...,S Face (High 122:3 Jan 1993) (Beghin accident)...,Nepal,2
3,ANN2,Annapurna II,None,7937,1,Annapurna-Damodar-Peri,True,False,False,None,None,2,Dyhrenfurth history 1960-1976 (MM 51:36-37 Sep...,N Face (MM 51:36 Sep 1976),Nepal,2
4,ANN3,Annapurna III,None,7555,1,Annapurna-Damodar-Peri,True,False,False,None,None,2,None,S Side (MM 125:11 Jan 1989)\r\nSW Face (MM 71:...,Nepal,2


In [12]:
df['local_names'] = df.pkname2.str.split(', ')

In [13]:
local_names_df = df[['local_names']]\
	.dropna()\
	.rename({'local_names': 'name'}, axis=1)\
	.explode('name')\
	.drop_duplicates(ignore_index=True)\
	.reset_index(names='id')

In [14]:
df.drop(['pkname2', 'local_names'], axis=1, inplace=True)

In [15]:
df.head()

,peakid,pkname,heightm,himal,region,open,unlisted,trekking,trekyear,restrict,pstatus,refermemo,photomemo,host,location_id
0,AMAD,Ama Dablam,6814,12,Khumbu-Rolwaling-Makalu,True,False,False,None,None,2,None,W Face (High 126:5 May 1993)\r\nSE Face (High ...,Nepal,0
1,AMPG,Amphu Gyabjen,5630,12,Khumbu-Rolwaling-Makalu,True,False,False,None,Opened in 2002,2,None,None,Nepal,1
2,ANN1,Annapurna I,8091,1,Annapurna-Damodar-Peri,True,False,False,None,None,2,Dyhrenfurth history 1950-1977 (MM 58:44-47 Nov...,S Face (High 122:3 Jan 1993) (Beghin accident)...,Nepal,2
3,ANN2,Annapurna II,7937,1,Annapurna-Damodar-Peri,True,False,False,None,None,2,Dyhrenfurth history 1960-1976 (MM 51:36-37 Sep...,N Face (MM 51:36 Sep 1976),Nepal,2
4,ANN3,Annapurna III,7555,1,Annapurna-Damodar-Peri,True,False,False,None,None,2,None,S Side (MM 125:11 Jan 1989)\r\nSW Face (MM 71:...,Nepal,2


In [16]:
df_local_names =

SyntaxError: invalid syntax (4286434455.py, line 1)